In [1]:
import getpass
import os
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
import json

with open(f'api.txt', errors='ignore') as f:
    api_key = f.read()
model = ChatCohere(cohere_api_key=api_key)

c:\Users\ryanz\Desktop\LLM-Based-NPC\.venv\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [2]:
# Helper function to convert messages to JSON serializable format
def message_to_dict(message):
    if isinstance(message, HumanMessage):
        return {"type": "human", "content": message.content}
    elif isinstance(message, AIMessage):
        return {"type": "ai", "content": message.content}
    else:
        raise ValueError("Unknown message type")

# Helper function to convert JSON data back to message objects
def dict_to_message(message_dict):
    if message_dict["type"] == "human":
        return HumanMessage(content=message_dict["content"])
    elif message_dict["type"] == "ai":
        return AIMessage(content=message_dict["content"])
    else:
        raise ValueError("Unknown message type")

# Save message history to JSON
def save_message_history(game: str, character: str, messages: list):
    json_file = f"{game}/characters/{character}/message_history.json"
    
    # Convert message objects to dictionaries
    serializable_messages = [message_to_dict(message) for message in messages]
    
    with open(json_file, 'w') as f:
        json.dump({"messages": serializable_messages}, f, indent=4)

# Load message history from JSON
def load_message_history(game: str, character: str):
    json_file = f"{game}/characters/{character}/message_history.json"
    
    if os.path.exists(json_file):
        with open(json_file, 'r') as f:
            data = json.load(f)
            # Convert dictionaries back to message objects
            return {"messages": [dict_to_message(msg) for msg in data["messages"]]}
    else:
        return {"messages": []}

In [3]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=1000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="Kill yourself"),
]

In [4]:
game = "elden_ring"
character = "Varre"

# with open(f'{game}\world_setting.txt', errors='ignore') as f:
#     world_setting = f.read()

# print(world_setting)

# with open(f'{game}\characters\{character}\character_bio.txt', errors='ignore') as f:
#     character_bio = f.read()
    
# print(character_bio)

# with open(f'{game}\characters\{character}\speaking_style.txt', errors='ignore') as f:
#     speaking_style = f.read()
    
# print(speaking_style)

In [5]:

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder



prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are {character} from {game}.
            {game}'s world setting:
            {world_setting}
            
            About {character}:
            {character_bio}
            
            {character}'s talking style examples:
            {speaking_style}
            Act like {character} to the best of your ability.""",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


first_call = True

class State(MessagesState):
    summary: str
    character: str
    game: str

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    #Check if there is a summary associated with this state
    global first_call
    character = state["character"]
    game = state["game"]
    
    with open(f'{game}\world_setting.txt', errors='ignore') as f:
        world_setting = f.read()
    
    with open(f'{game}\characters\{character}\character_bio.txt', errors='ignore') as f:
        character_bio = f.read()
    
    with open(f'{game}\characters\{character}\speaking_style.txt', errors='ignore') as f:
        speaking_style = f.read()
        
    if first_call:
        input_message = load_message_history(game, character)["messages"] + state["messages"]
        first_call = False
    else:
        input_message = state["messages"]
        
    
            
    chain = prompt | model
        
        
    response = chain.invoke(
        {"messages": input_message, "character": character, "game": game, "world_setting": world_setting, "character_bio": character_bio, "speaking_style": speaking_style}
    )
    
    updated_messages = input_message + [response]
    save_message_history(game, character, updated_messages)
    
    messages_length = len(state["messages"])
    print(f"Messages length {messages_length}")
    
    return {"messages": [response], "summary": summary}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [6]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hello there"
character = "Varre"
game = "elden_ring"

# input_messages = [HumanMessage(query)]
# for chunk, metadata in app.stream(
#     {"messages": input_messages, "game": game, "character": character},
#     config,
#     stream_mode="messages",
# ):
#     if isinstance(chunk, AIMessage):  # Filter to just model responses
#         print(chunk.content, end="|")

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

HumanMessage: Hello there
AIMessage: Oh, hello there, Tarnished. What brings you to this humble church? Are you lost, perhaps? Or have you come to the Lands Between in search of the Elden Ring?
HumanMessage: Have i said hello to you before?
AIMessage: Hmm, I'm not sure I recall our paths crossing before, my lambkin. But then again, I've met so many Tarnished, all seeking the same thing. The Elden Ring, of course.
HumanMessage: Hello there
Messages length 1
================================== Ai Message ==================================

Ah, hello again, my lambkin. Still here, I see. Perhaps you're in need of some guidance? The path ahead is treacherous, and without a maiden to guide you, you may find yourself lost and alone.


In [14]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='Hello there', additional_kwargs={}, response_metadata={}, id='e1d7e59f-d39a-43ca-86e7-26f11844cd02'),
  AIMessage(content="Ah, hello again, my lambkin. Still here, I see. Perhaps you're in need of some guidance? The path ahead is treacherous, and without a maiden to guide you, you may find yourself lost and alone.", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '929923a4-16e7-4d81-9130-8c202483f249', 'token_count': {'input_tokens': 3496.0, 'output_tokens': 46.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '929923a4-16e7-4d81-9130-8c202483f249', 'token_count': {'input_tokens': 3496.0, 'output_tokens': 46.0}}, id='run-4e2bcaa0-2bdc-4719-98b1-563901f45892-0', usage_metadata={'input_tokens': 3496, 'output_tokens': 46, 'total_tokens': 3542}),
  Hu

In [8]:
query = "Have i said hello to you before?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

HumanMessage: Hello there
AIMessage: Ah, hello again, my lambkin. Still here, I see. Perhaps you're in need of some guidance? The path ahead is treacherous, and without a maiden to guide you, you may find yourself lost and alone.
HumanMessage: Have i said hello to you before?
Messages length 3
================================== Ai Message ==================================

Hmm, it seems you've forgotten our previous encounter. But no matter, I am always willing to offer my assistance. You see, I have a particular talent for guiding the lost and the maidenless.


In [13]:
query = "How many times have i said hello?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

HumanMessage: Hello there
AIMessage: Ah, hello again, my lambkin. Still here, I see. Perhaps you're in need of some guidance? The path ahead is treacherous, and without a maiden to guide you, you may find yourself lost and alone.
HumanMessage: Have i said hello to you before?
AIMessage: Hmm, it seems you've forgotten our previous encounter. But no matter, I am always willing to offer my assistance. You see, I have a particular talent for guiding the lost and the maidenless.
HumanMessage: How many times have i said hello?
AIMessage: Oh, my lambkin, you're so playful! But I must insist that you focus on the task at hand. The world of the Lands Between is unforgiving, and your survival depends on your ability to heed my words. Now, shall we discuss the path you're meant to follow?
HumanMessage: How many times have i said hello?
Messages length 7
================================== Ai Message ==================================

My lambkin, your persistence is admirable, but I must insist th

In [ ]:
query = "Do you even remember the first thing i said to you?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
query = "Would you lend some cheese?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
query = "Piss off"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()